In [2]:
%%capture
pip install transformers

In [ ]:
# Used for Testing and prototyping

In [3]:
import pprint
from transformers import  AutoTokenizer
pp = pprint.PrettyPrinter(indent=4)

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base', model_max_length=512)

In [4]:
import itertools

def read_file(filename, top_k):
  doc = []
  with open(filename) as fileobject:
    top_2k = itertools.islice(fileobject, 2*top_k) 
    for line in top_2k:
          doc.append(line[:-1])
  return doc

doc = read_file('drive/MyDrive/Colab Notebooks/corpus/dev.txt', 2)

pp.pprint(doc)

[   'Alte mărci comerciale utilizate vreme îndelungată sunt Löwenbräu, '
    'deținătorii căreia spun că este folosită din 1383, și Stella Artois  din '
    '1366.',
    'Alte mărci comerciale utilizate vreme îndelungată sunt Löwenbräu, '
    'deținători căreia spun că este folosită din 1383, și Stella Artois  din '
    '1366.',
    'Cu aceeași grafie a fost săpată și inscripția, tocită însă de vreme, pe '
    'prestolul de piatră cu profile.',
    'Cu aceași grafie a fost săpata și inscripția, tocită însă de vreme, pe '
    'prestolul de piatră cu profile.']


In [5]:
def roberta_tokenize(doc):
  new_doc = []
  for i in range(0, len(doc), 2):
    new_doc.append((tokenizer.tokenize(doc[i]), tokenizer.tokenize(doc[i + 1])))  
  return new_doc


def print_pair(doc):
  for correct, wrong in doc:
    print(correct)
    print(wrong)
    print("")


token_doc = roberta_tokenize(doc)

In [6]:
def levenshtein(tokens1, tokens2):
  if len(tokens1) == 0: #len(t) inserari
    return len(tokens2)

  if len(tokens2) == 0: #len(t) inserari
    return len(tokens1)

  if tokens1[0] == tokens2[0]: #0 operatii
     return levenshtein(tokens1[1:], tokens2[1:])
  
  res1 = levenshtein(tokens1[1:], tokens2) #primul din token2 e sters

  res2 = levenshtein(tokens1, tokens2[1:]) #primul din token2 e adaugat

  res3 = levenshtein(tokens1[1:], tokens2[1:]) #primele sunt diferite

  return 1 + min(res1, res2, res3)

In [7]:
def add_opp(lst, opp, n):
  for i in range(n):
    lst.append(opp)
  return lst

def best_distance(res1, res2, res3):
  res1_d = len(res1)
  res2_d = len(res2)
  res3_d = len(res3)

  min_d = res1_d
  min = res1

  if res2_d < min_d:
     min_d = res2_d
     min = res2

  if res3_d < min_d:
     min_d = res3_d
     min = res3
    
  return min

def pseudo_levenshtein(tokens1, tokens2, ops):
  if len(tokens1) == 0: #len(t) inserari
    return add_opp(ops, 'add', len(tokens2))

  if len(tokens2) == 0: #len(t) inserari
    return add_opp(ops, 'del', len(tokens1))

  if tokens1[0] == tokens2[0]: #0 operatii
    return pseudo_levenshtein(tokens1[1:], tokens2[1:], ops)

  ops1 = ops.copy()
  ops1.append("del")
  ops2 = ops.copy()
  ops2.append("add")
  ops3 = ops.copy()
  ops3.append("dif")

  res1 = pseudo_levenshtein(tokens1[1:], tokens2, ops1) #primul din token2 e sters
  res2 = pseudo_levenshtein(tokens1, tokens2[1:], ops2) #primul din token2 e adaugat
  res3 = pseudo_levenshtein(tokens1[1:], tokens2[1:], ops3) #primele sunt diferite
  min = best_distance(res1, res2, res3)

  return min


pair1 = ["câștiguri", "căștiguri"]
pair2 = ["șeful departamentului politic", 
         "șef Dep. Politic"]
pair3 = ["nu merită", "nu se merită"]
pair4 = ["Omul negru nu are o înfățișare specifică, întrucât concepțiile privind chipul unui monstru variază de la o gospodărie la alta, chiar în cadrul aceleiași comunități.",
         "Omul negru nu are o înfățișare specifica, întrucât concepțile privind chipul unui monstru variază de la o gospodărie la alta, chiar în cadrul aceleași comunități."]
pair5 = ["Rămas-bun", "Rămas bun"]
pair6 = ["700 de milioane de euro", "700 milioane euro"]
pair7 = ["Poartă numele fizicianului englez P.A.M. Dirac  care a utilizat-o extensiv în formularea sa a mecanicii cuantice, dar prezența ei în matematică este mai veche și e de exemplu implicită în folosirea integralei Stieltjes.",
         "Poartă numele fizicianului englez P.A.M. Dirac  care a utilizato extensiv în formularea sa a mecanicii cuantice, dar prezența ei în matematică este mai veche și e de exemplu implicită în folosirea integralei Stieltjes."]
pair8 = ["copilul a prezentat afecțiuni respiratorii", "copilul a acuzat afecțiuni respiratorii"]
pair9 = ["Și vă întreb pe toți", "Și vă întreb cu toții"]

tokenized_pair = roberta_tokenize(pair1)

print(tokenized_pair[0][0])
print(tokenized_pair[0][1])

['▁câștig', 'uri']
['▁că', 'ști', 'guri']


In [8]:
import _pickle

def print_matrix(d):
  for row in d:
    print(row)

def findMin(d, i, s):
  min = d
  if i[0] < min[0]:
     min = i
  if s[0] < min[0]:
     min = s
  return min  

def diff_finder(source, target):
  
  lenS = len(source) + 1
  lenT = len(target) + 1

  d = [[[0, []] for x in range(lenT)] for y in range(lenS)] 
  for i in range(1, lenS):
    d[i][0][0] = i
    for k in range(i):
      d[i][0][1].append(('del', k, source[k]))

  for j in range(1, lenT):
    d[0][j][0] = j
    for k in range(j):
      d[0][j][1].append(('add', k, target[k]))

  for i in range(1, lenS):
    for j in range(1, lenT):
      if source[i - 1] == target[j - 1]:
        subCost = 0
      else:
        subCost = 1
      
      insert = _pickle.loads(_pickle.dumps(d[i][j - 1], -1)) #= 'add'

      insert[0] += 1
      insert[1].append(('add', i , target[j - 1]))
      
      delete = _pickle.loads(_pickle.dumps(d[i - 1][j], -1)) #= 'add'
      delete[0] += 1
      delete[1].append(('del', i - 1,  source[i - 1]))

      subst = _pickle.loads(_pickle.dumps(d[i-1][j-1], -1))

      if(subCost != 0):
        subst[0] += 1
        subst[1].append(('dif', i - 1 , (source[i - 1], target[j - 1])))

      min = findMin(delete, insert, subst)
      d[i][j] = min
  #print_matrix(d)
  return d[lenS - 1][lenT - 1]

#source = tokenized_pair[0][1]
#target = tokenized_pair[0][0]

source = tokenizer(pair1[1])
target = tokenizer(pair1[0])
print(source)
print(target)
print(diff_finder(source['input_ids'], target['input_ids']))

source = tokenized_pair[0][1]
target = tokenized_pair[0][0]
print(source)
print(target)
print(diff_finder(source, target))

{'input_ids': [0, 1362, 17566, 69140, 2], 'attention_mask': [1, 1, 1, 1, 1]}
{'input_ids': [0, 115567, 1162, 2], 'attention_mask': [1, 1, 1, 1]}
[3, [('dif', 1, (1362, 115567)), ('dif', 2, (17566, 1162)), ('del', 3, 69140)]]
['▁că', 'ști', 'guri']
['▁câștig', 'uri']
[3, [('dif', 0, ('▁că', '▁câștig')), ('dif', 1, ('ști', 'uri')), ('del', 2, 'guri')]]


In [16]:
import json

def correction_masks(wrong_token, diff):
  buckets_correction = [0] * len(wrong_token)
  buckets_add = [0] * (len(wrong_token) * 2)
  for correction in diff[1]:
    if correction[0] != 'add': #dif = 1, del = 2
      if correction[0] == 'dif':
        buckets_correction[correction[1]] = 1
      if correction[0] == 'del':
        buckets_correction[correction[1]] = 2
    else:
      buckets_add[correction[1]] += 1

  return [buckets_correction, buckets_add[0:len(wrong_token)]]


def count_corrections(right, wrong, file_new):

  right_token = tokenizer(right, truncation=True)
  wrong_token = tokenizer(wrong, truncation=True)

  diff = diff_finder(wrong_token['input_ids'], right_token['input_ids'])
  [correction_mask, add_mask] = correction_masks(wrong_token['input_ids'], diff)
  json_dump = {'input_ids':wrong_token['input_ids'], 
               'attention_mask' :wrong_token['attention_mask'], 
               'labels':correction_mask, 'add_mask':add_mask}

  file_new.write(json.dumps(json_dump))
  file_new.write('\n')

  return diff

def find_consecutive_adds(corections):
  consecutive_adds_count = 0
  consecutive_adds_idex = None
  max_count = 0
  if(corections[0] != 0):
    for correction in corections[1]:
      if correction[0] == 'add':
        if correction[1] != consecutive_adds_idex:
          consecutive_adds_count = 1
          consecutive_adds_idex = correction[1]
        else:
          consecutive_adds_count += 1
      else:
        max_count = consecutive_adds_count
        consecutive_adds_count = 0
        consecutive_adds_idex = None
  if consecutive_adds_count > max_count:
    max_count = consecutive_adds_count
  return max_count


def create_dataset(filename_old_dataset, filename_new_dataset, top_k):
  file_old = open(filename_old_dataset, "r")
  file_new = open(filename_new_dataset, "w")
  count = 0
  
  while True:
    count += 2
    if count > 2*top_k:
      break
    line_correct = file_old.readline()
    line_wrong = file_old.readline()
    if (not line_correct) or (not line_wrong):
      break
    diff = count_corrections(line_correct, line_wrong, file_new)
    max_adds_curr = find_consecutive_adds(diff)

  file_old.close()
  return

In [17]:
original_dataset = 'drive/MyDrive/Colab Notebooks/corpus/10_mil_dirty_clean_better.txt'
new_dataset = 'drive/MyDrive/Colab Notebooks/corpus/new_dataset.txt'

file_new = open(new_dataset, "w")
file_new.write('')
file_new.close()

create_dataset(original_dataset, new_dataset, 100000)

In [18]:
original_dataset = 'drive/MyDrive/Colab Notebooks/corpus/train.txt'
new_dataset = 'drive/MyDrive/Colab Notebooks/corpus/validation.txt'

file_new = open(new_dataset, "w")
file_new.write('')
file_new.close()

create_dataset(original_dataset, new_dataset, 10000)

In [ ]:
original_dataset = 'drive/MyDrive/Colab Notebooks/corpus/10_mil_dirty_clean_better.txt'
new_dataset = 'drive/MyDrive/Colab Notebooks/corpus/big_dataset.txt'

file_new = open(new_dataset, "w")
file_new.write('')
file_new.close()

create_dataset(original_dataset, new_dataset, 1000000)